In [1]:
!pip install fastapi motor pydantic[dotenv] uvicorn nest_asyncio

   ---------------------------------------- 0.0/94.7 kB ? eta -:--:--
   ----------------- ---------------------- 41.0/94.7 kB ? eta -:--:--
   ---------------------------------------- 94.7/94.7 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/74.8 kB ? eta -:--:--
   ---------------------------------------- 74.8/74.8 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/63.7 kB ? eta -:--:--
   ---------------------------------------- 63.7/63.7 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/924.9 kB ? eta -:--:--
   ----- ---------------------------------- 122.9/924.9 kB ? eta -:--:--
   ---------------- ----------------------- 378.9/924.9 kB 5.9 MB/s eta 0:00:01
   -------------------------------------- - 890.9/924.9 kB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 924.9/924.9 kB 6.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/73.3 kB ? eta -:--:--
   ----------------------------

In [19]:
# Use nest_asyncio for Running FastAPI
import nest_asyncio
nest_asyncio.apply()

In [21]:
from motor.motor_asyncio import AsyncIOMotorClient

MONGO_DB_URL = "mongodb+srv://Mansi:<Sample>@cluster0.1ktc5.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"
client = AsyncIOMotorClient(MONGO_DB_URL)
database = client.school_blog
blog_collection = database.get_collection("blogs")

In [23]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class BlogModel(BaseModel):
    id: Optional[str] = None
    title: str = Field(..., min_length=1, max_length=100)
    content: str = Field(..., min_length=1)
    author: str = Field(..., min_length=1, max_length=50)
    created_at: datetime = Field(default_factory=datetime.now)
    updated_at: datetime = Field(default_factory=datetime.now)

class UpdateBlogModel(BaseModel):
    title: Optional[str]
    content: Optional[str]
    author: Optional[str]
    updated_at: Optional[datetime] = None


In [27]:
from fastapi import FastAPI, HTTPException, Body
from bson import ObjectId
from datetime import datetime

app = FastAPI()

# Helper function to convert MongoDB objects to JSON-friendly format
def blog_helper(blog) -> dict:
    return {
        "id": str(blog["_id"]),
        "title": blog["title"],
        "content": blog["content"],
        "author": blog["author"],
        "created_at": blog.get("created_at"),
        "updated_at": blog.get("updated_at")
    }


In [29]:
# Add CRUD endpoints
# create a new blog post

@app.post("/blog", response_description="Add a new blog")
async def create_blog(blog: BlogModel = Body(...)):
    blog_data = blog.dict()
    blog_data["created_at"] = datetime.now()
    blog_data["updated_at"] = datetime.now()
    new_blog = await blog_collection.insert_one(blog_data)
    created_blog = await blog_collection.find_one({"_id": new_blog.inserted_id})
    return blog_helper(created_blog)



In [31]:
# GET all blogs

@app.get("/blogs", response_description="List all blogs")
async def get_blogs():
    blogs = []
    async for blog in blog_collection.find():
        blogs.append(blog_helper(blog))
    return blogs


In [33]:
# Get a Single Blog Post by ID

@app.get("/blog/{id}", response_description="Get a single blog by ID")
async def get_blog(id: str):
    blog = await blog_collection.find_one({"_id": ObjectId(id)})
    if blog:
        return blog_helper(blog)
    raise HTTPException(status_code=404, detail="Blog not found")


In [ ]:
# Update a Blog Post by ID

@app.put("/blog/{id}", response_description="Update a blog")
async def update_blog(id: str, blog: UpdateBlogModel):
    updated_blog = {k: v for k, v in blog.dict().items() if v is not None}
    if updated_blog:
        result = await blog_collection.update_one({"_id": ObjectId(id)}, {"$set": updated_blog})
        if result.modified_count == 1:
            updated_blog = await blog_collection.find_one({"_id": ObjectId(id)})
            return blog_helper(updated_blog)
    raise HTTPException(status_code=404, detail="Blog not found")

In [ ]:
# Delete a Blog Post by ID

@app.delete("/blog/{id}", response_description="Delete a blog")
async def delete_blog(id: str):
    result = await blog_collection.delete_one({"_id": ObjectId(id)})
    if result.deleted_count == 1:
        return {"message": "Blog deleted successfully"}
    raise HTTPException(status_code=404, detail="Blog not found")


In [ ]:
import uvicorn

# Run FastAPI 
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)